In [1]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output

In [2]:
# start frozen lake env
env = gym.make("FrozenLake-v0")

In [3]:
# create the Q-table
action_space_size = env.action_space.n
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size,action_space_size))

In [4]:
# let's see q_table
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
# initializting Q-learning parameters
num_episodes = 10000 # play the game 10000 times
max_steps_per_episode = 100 # every game just play 100 steps. even not win or lose

learning_rate = 0.1 # we just use the information 10% of each time
discount_rate = 0.99 # what we think about the furture

exploration_rate = 1 # at first we should make exploration
max_exploration_rate = 1 # we can't pass over 1 becourse it's not mean anythihng
min_exploration_rate = 0.01 # even we get all the information , we should still exploration
exploration_decay_rate = 0.01 # well ,after 100 times we will just use the min I don't know if it is all right